In [6]:
import numpy as np
import os
from scipy import stats
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import chain, combinations

In [2]:
#!/usr/bin/env python

# Copyright 2016-2020 Biomedical Imaging Group Rotterdam, Departments of
# Medical Informatics and Radiology, Erasmus MC, Rotterdam, The Netherlands
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Adopted from https://github.com/yandexdataschool/roc_comparison."""

import numpy as np
import scipy.stats


# AUC comparison adapted from
# https://github.com/Netflix/vmaf/

def compute_midrank(x):
    """Computes midranks.

    Args:
       x - a 1D numpy array
    Returns:
       array of midranks

    """
    J = np.argsort(x)
    Z = x[J]
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = 0.5*(i + j - 1)
        i = j
    T2 = np.empty(N, dtype=float)
    # Note(kazeevn) +1 is due to Python using 0-based indexing
    # instead of 1-based in the AUC formula in the paper
    T2[J] = T + 1
    return T2


def compute_midrank_weight(x, sample_weight):
    """Computes midranks.

    Args:
       x - a 1D numpy array
    Returns:
       array of midranks

    """
    J = np.argsort(x)
    Z = x[J]
    cumulative_weight = np.cumsum(sample_weight[J])
    N = len(x)
    T = np.zeros(N, dtype=float)
    i = 0
    while i < N:
        j = i
        while j < N and Z[j] == Z[i]:
            j += 1
        T[i:j] = cumulative_weight[i:j].mean()
        i = j
    T2 = np.empty(N, dtype=float)
    T2[J] = T
    return T2


def fastDeLong(predictions_sorted_transposed, label_1_count):
    """Fast DeLong test computation.

    The fast version of DeLong's method for computing the covariance of
    unadjusted AUC.
    Args:
       predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
          sorted such as the examples with label "1" are first
    Returns:
       (AUC value, DeLong covariance)
    Reference:
     @article{sun2014fast,
       title={Fast Implementation of DeLong's Algorithm for
              Comparing the Areas Under Correlated Receiver Oerating Characteristic Curves},
       author={Xu Sun and Weichao Xu},
       journal={IEEE Signal Processing Letters},
       volume={21},
       number={11},
       pages={1389--1393},
       year={2014},
       publisher={IEEE}
     }

    """
    # Short variables are named as they are in the paper
    m = label_1_count
    n = predictions_sorted_transposed.shape[1] - m
    positive_examples = predictions_sorted_transposed[:, :m]
    negative_examples = predictions_sorted_transposed[:, m:]
    k = predictions_sorted_transposed.shape[0]

    tx = np.empty([k, m], dtype=float)
    ty = np.empty([k, n], dtype=float)
    tz = np.empty([k, m + n], dtype=float)
    for r in tqdm(range(k)):
        tx[r, :] = compute_midrank(positive_examples[r, :])
        ty[r, :] = compute_midrank(negative_examples[r, :])
        tz[r, :] = compute_midrank(predictions_sorted_transposed[r, :])
    aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
    v01 = (tz[:, :m] - tx[:, :]) / n
    v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
    sx = np.cov(v01)
    sy = np.cov(v10)
    delongcov = sx / m + sy / n
    return aucs, delongcov


def calc_pvalue(aucs, sigma):
    """Computes log(10) of p-values.

    Args:
       aucs: 1D array of AUCs
       sigma: AUC DeLong covariances
    Returns:
       log10(pvalue)

    """
    l = np.array([[1, -1]])
    z = np.abs(np.diff(aucs)) / (np.sqrt(np.dot(np.dot(l, sigma), l.T)))
    log10pval = np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)
    return np.power(10, log10pval)


def compute_ground_truth_statistics(ground_truth, sample_weight=None):
    assert np.array_equal(np.unique(ground_truth), [0, 1])
    order = (-ground_truth).argsort()
    label_1_count = int(ground_truth.sum())
    if sample_weight is None:
        ordered_sample_weight = None
    else:
        ordered_sample_weight = sample_weight[order]

    return order, label_1_count, ordered_sample_weight


def delong_roc_variance(ground_truth, predictions):
    """Computes ROC AUC variance for a single set of predictions.

    Args:
       ground_truth: np.array of 0 and 1
       predictions: np.array of floats of the probability of being class 1

    """
    sample_weight = None
    order, label_1_count, ordered_sample_weight = compute_ground_truth_statistics(
        ground_truth, sample_weight)
    predictions_sorted_transposed = predictions[np.newaxis, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count, ordered_sample_weight)
    assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
    return aucs[0], delongcov


def delong_roc_test(ground_truth, predictions_one, predictions_two):
    """Computes log(p-value) for hypothesis that two ROC AUCs are different.

    Args:
       ground_truth: np.array of 0 and 1
       predictions_one: predictions of the first model,
          np.array of floats of the probability of being class 1
       predictions_two: predictions of the second model,
          np.array of floats of the probability of being class 1

    """
    order, label_1_count, _ = compute_ground_truth_statistics(ground_truth)
    predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
    aucs, delongcov = fastDeLong(predictions_sorted_transposed, label_1_count)
    return calc_pvalue(aucs, delongcov)


def calc_delong(y_true, y_p, alpha=0.95):
    
#     alpha = .95

    y_true = y_true.reshape(-1)
    y_p = y_p.reshape(-1)

    auc, auc_cov = delong_roc_variance(
        y_true,
        y_p)

    auc_std = np.sqrt(auc_cov)
    lower_upper_q = np.abs(np.array([0, 1]) - (1 - alpha) / 2)

    ci = stats.norm.ppf(
        lower_upper_q,
        loc=auc,
        scale=auc_std)

    ci[ci > 1] = 1

    print('AUC:', auc)
    print('AUC COV:', auc_cov)
    print('95% AUC CI:', ci)
    
    return auc, auc_cov, ci

In [3]:
! ls

check_invalid_ecg.ipynb		   proposed
data_check_with_ori.ipynb	   proposed_cmc
data_processing			   proposed_limited_labeled
deepbeat_baseline		   __pycache__
deepmi				   result_plots
deepmi_limited_labels		   results_plot.ipynb
ecg_only_baseline		   run_stat_sig.py
ecg_only_baseline_limited_labeled  sample_ecg.npy
model_card.md			   sample_ppg.npy
output2(w_correction).txt	   stat_significance_test.ipynb
output.txt			   stat_significance_test.py
ppg_only_baseline		   tmp.ipynb
ppg_only_baseline_limited_labeled


In [4]:
testing_data_label_pairs = [
    ['data_simband_ecg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_simband_ppg_2400.npy', 'label_simband.npy', '/labs/hulab/Robust_learning_TESTDATA/simband_patient_idx_dict.pkl'],
    ['data_ucla_ecg_2400.npy', 'label_ucla_ecg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_ucla_ppg_2400.npy', 'label_ucla_ppg.npy', '/labs/hulab/Robust_learning_TESTDATA/UCLA_patient_idx_dict.pkl'],
    ['data_staford_2400.npy', 'label_staford.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_goodquality_2400.npy', 'label_staford_goodquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl'],
#     ['data_staford_badquality_2400.npy', 'label_staford_badquality.npy', '/labs/hulab/Robust_learning_TESTDATA/stanford_patient_idx_dict.pkl']
]

# PPG Datasets (full)

In [33]:
folders = ['simsiam', 'deepmi', 'deepbeat_baseline', 'ppg_only_baseline']
model_names = ['simsiam', 'deepmi', 'deepbeat', 'ppgonly']
dataset_lengths = ['2400', '2400', '800', '2400']
combs = list(combinations(folders, 2))
print(combs)

[('simsiam', 'deepmi'), ('simsiam', 'deepbeat_baseline'), ('simsiam', 'ppg_only_baseline'), ('deepmi', 'deepbeat_baseline'), ('deepmi', 'ppg_only_baseline'), ('deepbeat_baseline', 'ppg_only_baseline')]


In [21]:
for pair in testing_data_label_pairs:
        
    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ppg' in dataset or 'staford' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            print(targets.shape, pred_probs_1.shape, pred_probs_2.shape, auroc_1.shape, auroc_2.shape, auprc_1.shape, auprc_2.shape)
            p_auroc = delong_roc_test(targets, pred_probs_1, pred_probs_2)[0][0]
            p_auprc = stats.ttest_rel(auprc_1, auprc_2).pvalue
            print(f'{model_pair}: p_auroc={p_auroc} , p_auprc={p_auprc}')    

---------------------------------------------------------------
---------------------------------------------------------------
data_simband_ppg_2400
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)


100%|██████████| 2/2 [00:02<00:00,  1.07s/it]


('simsiam', 'deepmi'): p_auroc=0.0 , p_auprc=9.127480280470798e-194
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)


100%|██████████| 2/2 [00:02<00:00,  1.06s/it]


('simsiam', 'deepbeat_baseline'): p_auroc=0.0 , p_auprc=0.0
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)


...

100%|██████████| 2/2 [00:54<00:00, 27.05s/it]


('simsiam', 'deepbeat_baseline'): p_auroc=4.890782205998843e-125 , p_auprc=0.04602517483857248
(17556623,) (17556623,) (17556623,) (1000,) (1000,) (1000,) (1000,)


100%|██████████| 2/2 [00:56<00:00, 28.23s/it]


('simsiam', 'ppg_only_baseline'): p_auroc=0.0 , p_auprc=0.0
(17556623,) (17556623,) (17556623,) (1000,) (1000,) (1000,) (1000,)


KeyboardInterrupt: 

In [55]:
for pair in testing_data_label_pairs:
        
    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ppg' in dataset or 'staford' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
#             print(targets.shape, pred_probs_1.shape, pred_probs_2.shape, auroc_1.shape, auroc_2.shape, auprc_1.shape, auprc_2.shape)
            p_auroc_wc = stats.wilcoxon(auroc_1, auroc_2).pvalue
            p_auprc_wc = stats.wilcoxon(auprc_1, auprc_2).pvalue
        
            print(f'{model_pair}: p_auroc={p_auroc_wc} , p_auprc={p_auprc_wc}')        
            
#             x, y = auroc_1, auroc_2
#             d = x - y
#             d = np.compress(np.not_equal(d, 0), d)
#             count = len(d)
#             r = stats.rankdata(abs(d))
#             r_plus = np.sum((d > 0) * r)
#             r_minus = np.sum((d < 0) * r)
            
#             T = min(r_plus, r_minus)
            
#             mn = count * (count + 1.) * 0.25
#             se = count * (count + 1.) * (2. * count + 1.)
            
#             replist, repnum = stats.find_repeats(r)
#             if repnum.size != 0:
#                 # Correction for repeated elements.
#                 se -= 0.5 * (repnum * (repnum * repnum - 1)).sum()

#             se = np.sqrt(se / 24)

#             # apply continuity correction if applicable
#             d = 0
            
#             z = (T - mn - d) / se
#             prob = 2. * stats.distributions.norm.sf(abs(z))
            
#             z = z
            
            
#             print(prob)
            
            
#             break
            
            

---------------------------------------------------------------
---------------------------------------------------------------
data_simband_ppg_2400
('simsiam', 'deepmi'): p_auroc=7.004012126930545e-164 , p_auprc=4.882530023580109e-143
('simsiam', 'deepbeat_baseline'): p_auroc=4.654951174852043e-165 , p_auprc=9.36400998312439e-165
('simsiam', 'ppg_only_baseline'): p_auroc=3.510543381233979e-165 , p_auprc=5.492971800093215e-160
('deepmi', 'deepbeat_baseline'): p_auroc=1.0721500110969205e-115 , p_auprc=5.643844824027882e-153
('deepmi', 'ppg_only_baseline'): p_auroc=8.447931670801826e-146 , p_auprc=2.4812949873078955e-56
('deepbeat_baseline', 'ppg_only_baseline'): p_auroc=1.701765361433702e-34 , p_auprc=1.9795227674018953e-141
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ppg_2400
('simsiam', 'deepmi'): p_auroc=1.6220970106420827e-38 , p_auprc=2.7555357760491723e-36
('simsiam', 'deepbeat_baseline'

# ECG datasets (full)

In [57]:
folders = ['simsiam', 'deepmi', 'ecg_only_baseline']
model_names = ['simsiam', 'deepmi', 'ecgonly']
dataset_lengths = ['2400', '2400', '2400']
combs = list(combinations(folders, 2))
print(combs)

[('simsiam', 'deepmi'), ('simsiam', 'ecg_only_baseline'), ('deepmi', 'ecg_only_baseline')]


In [8]:
for pair in testing_data_label_pairs:
        
    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ecg' in dataset:
        print(dataset)
        for model_pair in combs:
            print(model_pair)
            
            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
#             print(targets.shape, pred_probs_1.shape, pred_probs_2.shape, auroc_1.shape, auroc_2.shape, auprc_1.shape, auprc_2.shape)
            p_auroc = delong_roc_test(targets, pred_probs_1, pred_probs_2)[0][0]
#             p_auroc = stats.ttest_rel(auroc_1, auroc_2).pvalue
            p_auprc = stats.ttest_rel(auprc_1, auprc_2).pvalue
            
            print(f'{model_pair}: p_auroc={p_auroc} , p_auprc={p_auprc}')
            
            

---------------------------------------------------------------
data_simband_ecg_2400
('simsiam', 'deepmi')


100%|██████████| 2/2 [00:01<00:00,  1.49it/s]


('simsiam', 'deepmi'): p_auroc=0.0 , p_auprc=0.0
('simsiam', 'ecg_only_baseline')


100%|██████████| 2/2 [00:01<00:00,  1.49it/s]


('simsiam', 'ecg_only_baseline'): p_auroc=0.0 , p_auprc=0.0
('deepmi', 'ecg_only_baseline')


...

('simsiam', 'deepmi'): p_auroc=0.0 , p_auprc=4.0864597254483e-98
('simsiam', 'ecg_only_baseline')


100%|██████████| 2/2 [09:17<00:00, 278.52s/it]


('simsiam', 'ecg_only_baseline'): p_auroc=0.0 , p_auprc=1.9704262934622083e-216
('deepmi', 'ecg_only_baseline')


100%|██████████| 2/2 [09:34<00:00, 287.17s/it]


('deepmi', 'ecg_only_baseline'): p_auroc=0.0 , p_auprc=1.0927738265027907e-152
---------------------------------------------------------------
---------------------------------------------------------------


In [58]:


for pair in testing_data_label_pairs:
        
    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ecg' in dataset:
        print(dataset)
        for model_pair in combs:
            print(model_pair)
            
            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            
            p_auroc_wc = stats.wilcoxon(auroc_1, auroc_2).pvalue
            p_auprc_wc = stats.wilcoxon(auprc_1, auprc_2).pvalue
        
            print(f'{model_pair}: p_auroc={p_auroc_wc} , p_auprc={p_auprc_wc}') 
        


---------------------------------------------------------------
data_simband_ecg_2400
('simsiam', 'deepmi')
('simsiam', 'deepmi'): p_auroc=3.5548037607056008e-127 , p_auprc=1.9748820424982348e-164
('simsiam', 'ecg_only_baseline')
('simsiam', 'ecg_only_baseline'): p_auroc=2.019183825066748e-73 , p_auprc=3.325859118934513e-165
('deepmi', 'ecg_only_baseline')
('deepmi', 'ecg_only_baseline'): p_auroc=4.419200071432701e-08 , p_auprc=1.5172038211816667e-164
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ecg_2400
('simsiam', 'deepmi')
('simsiam', 'deepmi'): p_auroc=9.980326940189327e-134 , p_auprc=7.264207136436138e-88
('simsiam', 'ecg_only_baseline')
('simsiam', 'ecg_only_baseline'): p_auroc=3.643623307702546e-162 , p_auprc=4.7522710116359023e-144
('deepmi', 'ecg_only_baseline')
('deepmi', 'ecg_only_baseline'): p_auroc=4.88392024197109e-165 , p_auprc=1.4544668682991088e-117
----------------------------

# PPG (limited)

In [59]:
folders = ['simsiam_limited_labeled', 'deepmi_limited_labels', 'ppg_only_baseline_limited_labeled']
model_names = ['simsiamlimitedlabels', 'deepmilimitedlabels', 'ppgonlylimitedlabel']
dataset_lengths = ['2400', '2400', '2400']
combs = list(combinations(folders, 2))
print(combs)

[('simsiam_limited_labeled', 'deepmi_limited_labels'), ('simsiam_limited_labeled', 'ppg_only_baseline_limited_labeled'), ('deepmi_limited_labels', 'ppg_only_baseline_limited_labeled')]


In [9]:
for pair in testing_data_label_pairs:

    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ppg' in dataset or 'staford' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            print(targets.shape, pred_probs_1.shape, pred_probs_2.shape, auroc_1.shape, auroc_2.shape, auprc_1.shape, auprc_2.shape)
#             p_auroc = delong_roc_test(targets, pred_probs_1, pred_probs_2)[0][0]
# #             p_auroc = stats.ttest_rel(auroc_1, auroc_2).pvalue
#             p_auprc = stats.ttest_rel(auprc_1, auprc_2).pvalue
            
#             print(f'{model_pair}: p_auroc={p_auroc} , p_auprc={p_auprc}')

---------------------------------------------------------------
---------------------------------------------------------------
data_simband_ppg_2400
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ppg_2400
(250683755,) (250683755,) (250683755,) (1000,) (1000,) (1000,) (1000,)
(250683755,) (250683755,) (250683755,) (1000,) (1000,) (1000,) (1000,)
(250683755,) (250683755,) (250683755,) (1000,) (1000,) (1000,) (1000,)
---------------------------------------------------------------
data_staford_2400
(17556623,) (17556623,) (17556623,) (1000,) (1000,) (1000,) (1000,)
(17556623,) (17556623,) (17556623,) (1000,) (1000,) (1000,) (1000,)
(17556623,) (17556623,) (17556623,) (1000,) (1000,) (1000,) (1000,)


In [61]:
for pair in testing_data_label_pairs:

    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ppg' in dataset or 'staford' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            p_auroc_wc = stats.wilcoxon(auroc_1, auroc_2).pvalue
            p_auprc_wc = stats.wilcoxon(auprc_1, auprc_2).pvalue
        
            print(f'{model_pair}: p_auroc={p_auroc_wc} , p_auprc={p_auprc_wc}')

---------------------------------------------------------------
---------------------------------------------------------------
data_simband_ppg_2400
('simsiam_limited_labeled', 'deepmi_limited_labels'): p_auroc=3.325859118934513e-165 , p_auprc=3.325859118934513e-165
('simsiam_limited_labeled', 'ppg_only_baseline_limited_labeled'): p_auroc=3.325859118934513e-165 , p_auprc=3.4169553748281145e-165
('deepmi_limited_labels', 'ppg_only_baseline_limited_labeled'): p_auroc=2.2605188352998174e-150 , p_auprc=3.325859118934513e-165
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ppg_2400
('simsiam_limited_labeled', 'deepmi_limited_labels'): p_auroc=3.325859118934513e-165 , p_auprc=3.325859118934513e-165
('simsiam_limited_labeled', 'ppg_only_baseline_limited_labeled'): p_auroc=3.325859118934513e-165 , p_auprc=3.325859118934513e-165
('deepmi_limited_labels', 'ppg_only_baseline_limited_labeled'): p_auroc=7.636

# ECG Limited

In [62]:
folders = ['deepmi_limited_labels', 'ecg_only_baseline_limited_labeled']
model_names = ['deepmilimitedlabels', 'ecgonlylimitedlabel']
dataset_lengths = ['2400', '2400']
combs = list(combinations(folders, 2))
print(combs)

[('deepmi_limited_labels', 'ecg_only_baseline_limited_labeled')]


In [63]:
for pair in testing_data_label_pairs:

    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ecg' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            print(targets.shape, pred_probs_1.shape, pred_probs_2.shape, auroc_1.shape, auroc_2.shape, auprc_1.shape, auprc_2.shape)
            p_auroc = delong_roc_test(targets, pred_probs_1, pred_probs_2)[0][0]
            p_auroc = stats.ttest_rel(auroc_1, auroc_2).pvalue
            print(np.cov(auprc_1, auprc_2))
            print(np.isnan(auprc_1).any(), np.isnan(auprc_2).any())
            p_auprc = stats.ttest_rel(auprc_1, auprc_2).pvalue
            
            print(f'{model_pair}: p_auroc={p_auroc} , p_auprc={p_auprc}')
        
            

---------------------------------------------------------------
data_simband_ecg_2400
(856988,) (856988,) (856988,) (1000,) (1000,) (1000,) (1000,)


100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


[[0.00060045 0.0008392 ]
 [0.0008392  0.05907573]]
False False
('deepmi_limited_labels', 'ecg_only_baseline_limited_labeled'): p_auroc=5.908506151078022e-97 , p_auprc=9.951774711370074e-219
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ecg_2400
(250683755,) (250683755,) (250683755,) (1000,) (1000,) (1000,) (1000,)


100%|██████████| 2/2 [09:37<00:00, 288.70s/it]


[[0.00032977 0.00083913]
 [0.00083913 0.00375018]]
False False
('deepmi_limited_labels', 'ecg_only_baseline_limited_labeled'): p_auroc=1.1920205966593885e-86 , p_auprc=0.0
---------------------------------------------------------------
---------------------------------------------------------------


In [64]:
for pair in testing_data_label_pairs:

    dataset = pair[0].split('.')[0]
    print('---------------------------------------------------------------')
    if 'ecg' in dataset:
        print(dataset)
        for model_pair in combs:

            dataset_1 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[0])])
            dataset_2 = dataset.replace('2400', dataset_lengths[folders.index(model_pair[1])])
            
            targets = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_targets.npy')
            pred_probs_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_pred_probs.npy')
            pred_probs_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_pred_probs.npy')
            auroc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_aurocs.npy')       
            auroc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_aurocs.npy')
            auprc_1 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_1}_{model_names[folders.index(model_pair[0])]}_auprcs.npy')            
            auprc_2 = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
            p_auroc_wc = stats.wilcoxon(auroc_1, auroc_2).pvalue
            p_auprc_wc = stats.wilcoxon(auprc_1, auprc_2).pvalue
        
            print(f'{model_pair}: p_auroc={p_auroc_wc} , p_auprc={p_auprc_wc}')

---------------------------------------------------------------
data_simband_ecg_2400
('deepmi_limited_labels', 'ecg_only_baseline_limited_labeled'): p_auroc=9.930811414707158e-83 , p_auprc=2.467723781036517e-151
---------------------------------------------------------------
---------------------------------------------------------------
data_ucla_ecg_2400
('deepmi_limited_labels', 'ecg_only_baseline_limited_labeled'): p_auroc=6.5387702296231955e-77 , p_auprc=3.325859118934513e-165
---------------------------------------------------------------
---------------------------------------------------------------


In [10]:
print(dataset_2, model_names[folders.index(model_pair[1])])
auprc_a = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs.npy')
auprc_b = np.load(f'/labs/hulab/stark_stuff/ppg_ecg_project/data/{dataset_2}_{model_names[folders.index(model_pair[1])]}_auprcs22.npy')

data_simband_ecg_2400 ecgonlylimitedlabel


In [12]:
print(np.subtract(auprc_a, auprc_b))


[-0.12736596  0.1113902   0.097986   -0.12026541  0.10863852  0.08641139
  0.08410832  0.13772868  0.11613212 -0.11047159  0.10411925  0.09066356
 -0.11684781 -0.14474963 -0.10132021 -0.11856603  0.1079938   0.10961868
  0.09353498 -0.11655884  0.09695574  0.09675599  0.0954865   0.08374276
  0.09695494  0.09538618 -0.13222351  0.08940818  0.07510125 -0.16161896
 -0.10932272 -0.13019148 -0.12763121 -0.16304358  0.08529244  0.11625032
  0.0910924   0.10116565 -0.13310669  0.10303141  0.10516644  0.09491135
  0.10445315  0.08400382  0.08765855  0.11400105 -0.1207005   0.1111073
 -0.13830953  0.10739836 -0.11107001  0.10330599 -0.1352469  -0.15405091
  0.10597005 -0.1365743   0.0681474  -0.12848281 -0.14936784 -0.13283836
 -0.14560917 -0.13523861  0.11133322  0.09620434  0.09164903  0.0981501
 -0.14205062  0.09048466  0.09618169 -0.13573429  0.09120903  0.08723406
  0.09562974  0.10111554 -0.1199902   0.10194261  0.10320011 -0.11931961
  0.08801583 -0.12856544  0.09793015  0.09127223  0.1